In [1]:
!pip install transformers

     |████████████████████████████████| 573kB 38.1MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 3.7MB 46.0MB/s 
     |████████████████████████████████| 1.0MB 43.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=1ba8a9fa0005e0e887f693b60087ef530c0489efe11d264e4d910d56fda499bc
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
%cd /content/gdrive/Shared\ drives/si630_hw5

import pandas as pd

def read_data(filename):
    with open(filename) as f:
        data = {}
        headers = f.readline().strip().split("\t")
        for h in headers:
            data[h] = []
        line = f.readline()
        while line:
            pos = line.find("\t")
            data[headers[0]].append(line[:pos])
            data[headers[1]].append(line[pos+1:-1])
            line = f.readline()
    return pd.DataFrame(data)

/content/gdrive/Shared drives/si630_hw5


In [4]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


In [5]:
!pwd

/content/gdrive/Shared drives/si630_hw5


In [0]:
#1 fine tune with distilbert model
# %%capture
# !python run_language_modeling.py \
#     --output_dir=./output_Theo_cls_uncased_new \
#     --model_type=distilbert-base-uncased \
#     --model_name_or_path=distilbert-base-uncased \
#     --do_train \
#     --train_data_file=./data/generation/train.txt \
#     --do_eval \
#     --mlm \
#     --eval_data_file=./data/generation/dev.txt \
#     --line_by_line

# we cannot train normal bert model since there is not enough memory

In [6]:
# Load the training dataset into a pandas dataframe.
df = read_data('./data/classification/train.tsv')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 115,590



,class,text
64290,machine,the film is a mid - sized conspiracy plot set ...
38427,human,"in san francisco, well-known sportsman kay hoo..."
89215,machine,"having recently returned to thailand, the chil..."
107671,machine,lt. the story concerns a hopeful and confused ...
76151,machine,"mookaiyah comes to bangalore, never intending ..."
2154,human,the cartoon starts with a brief narration desc...
92233,machine,"lettice reeth is a musician, who discovers the..."
2517,human,the two friends were getting near the end of t...
91294,machine,*chapter 1 - commander anne davis has a mental...
63145,machine,"at the outset of the novel, jack learns of a d..."


In [0]:
import numpy as np

In [0]:
# Get the lists of sentences and their labels.
sentences = df['text'].values
labels = df['class'].values

In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Loading BERT tokenizer...


In [0]:
labels[labels == 'machine'] = 0
labels[labels == 'human'] = 1

In [0]:
labels = labels.astype(dtype = 'int')

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  once upon a time in the southern chinese city of guancheng, there lived four families, each of them faithful keepers of martial arts. anyone who wanted to establish a new sect, or
Token IDs: tensor([  101,  2320,  2588,  1037,  2051,  1999,  1996,  2670,  2822,  2103,
         1997, 19739,  2319, 21043,  1010,  2045,  2973,  2176,  2945,  1010,
         2169,  1997,  2068, 11633, 24018,  1997,  7761,  2840,  1012,  3087,
         2040,   102])


In [0]:
# Load the validation dataset into a pandas dataframe.
df_val = read_data('./data/classification/dev.tsv')
df_test = read_data('./data/classification/test.tsv')

# Report the number of sentences.
print('Number of validation sentences: {:,}\n'.format(df_val.shape[0]))
print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))
# Display 10 random rows from the data.
# df_val.sample(10)

Number of validation sentences: 14,462

Number of test sentences: 14,434



In [0]:
# pre-processing for validation set
sentences_val = df_val['text'].values
labels_val = df_val['class'].values

labels_val[labels_val == 'machine'] = 0
labels_val[labels_val == 'human'] = 1

labels_val = labels_val.astype(dtype = 'int')

sentences_test = df_test['text'].values
labels_test = df_test['class'].values

labels_test[labels_test == 'machine'] = 0
labels_test[labels_test == 'human'] = 1

labels_test = labels_test.astype(dtype = 'int')
labels_test

array([1, 1, 1, ..., 0, 0, 0])

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_val = []
attention_masks_val = []

# For every sentence...
for sent in sentences_val:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_val.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_val.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val = torch.cat(attention_masks_val, dim=0)
labels_val = torch.tensor(labels_val)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_val[0])
print('Token IDs:', input_ids_val[0])


input_ids_test = []
attention_masks_test = []

# For every sentence...
for sent in sentences_test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_test.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
labels_test = torch.tensor(labels_test)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_test[0])
print('Token IDs:', input_ids_test[0])

Original:  i continued pleading with my parents to cancel our flight to bangladesh or at least leave me at home. the last time i visited bangladesh, i was 6. i remembered getting sick
Token IDs: tensor([  101,  1045,  2506, 16418,  2007,  2026,  3008,  2000, 17542,  2256,
         3462,  2000,  7269,  2030,  2012,  2560,  2681,  2033,  2012,  2188,
         1012,  1996,  2197,  2051,  1045,  4716,  7269,  1010,  1045,  2001,
         1020,   102])
Original:  kenji koiso is a young high school student with a gift in mathematics and a part-time moderator in the massive computer-simulated virtual reality world oz along with his friend takashi sakuma .
Token IDs: tensor([  101, 25894, 12849, 19565,  2003,  1037,  2402,  2152,  2082,  3076,
         2007,  1037,  5592,  1999,  5597,  1998,  1037,  2112,  1011,  2051,
        29420,  1999,  1996,  5294,  3274,  1011, 23599,  7484,  4507,  2088,
        11472,   102])


In [0]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(input_ids, attention_masks, labels)
val_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)
test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [10]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # # ========================================
    # #               Training
    # # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    # training_stats.append(
    #     {
    #         'epoch': epoch_i + 1,
    #         'Training Loss': avg_train_loss,
    #         'Valid. Loss': avg_val_loss,
    #         'Valid. Accur.': avg_val_accuracy,
    #         'Training Time': training_time,
    #         'Validation Time': validation_time
    #     }
    # )

    # ========================================
    #              Test
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Test...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_test_accuracy = total_eval_accuracy / len(test_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_eval_loss / len(test_dataloader)
    
    # Measure how long the validation run took.
    test_time = format_time(time.time() - t0)
    
    print("  test Loss: {0:.2f}".format(avg_test_loss))
    print("  test took: {:}".format(test_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Test. Loss': avg_test_loss,
            'Test. Accur.': avg_test_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )


print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of  3,613.    Elapsed: 0:00:05.
  Batch    80  of  3,613.    Elapsed: 0:00:11.
  Batch   120  of  3,613.    Elapsed: 0:00:16.
  Batch   160  of  3,613.    Elapsed: 0:00:21.
  Batch   200  of  3,613.    Elapsed: 0:00:26.
  Batch   240  of  3,613.    Elapsed: 0:00:32.
  Batch   280  of  3,613.    Elapsed: 0:00:37.
  Batch   320  of  3,613.    Elapsed: 0:00:42.
  Batch   360  of  3,613.    Elapsed: 0:00:47.
  Batch   400  of  3,613.    Elapsed: 0:00:53.
  Batch   440  of  3,613.    Elapsed: 0:00:58.
  Batch   480  of  3,613.    Elapsed: 0:01:03.
  Batch   520  of  3,613.    Elapsed: 0:01:08.
  Batch   560  of  3,613.    Elapsed: 0:01:14.
  Batch   600  of  3,613.    Elapsed: 0:01:19.
  Batch   640  of  3,613.    Elapsed: 0:01:24.
  Batch   680  of  3,613.    Elapsed: 0:01:29.
  Batch   720  of  3,613.    Elapsed: 0:01:35.
  Batch   760  of  3,613.    Elapsed: 0:01:40.
  Batch   800  of  3,613.    Elapsed: 0:01:45.
  Batch   840  of

In [0]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save_theo/cls_new'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to ./model_save_theo/cls_new


('./model_save_theo/cls_new/vocab.txt',
 './model_save_theo/cls_new/special_tokens_map.json',
 './model_save_theo/cls_new/added_tokens.json')

Test on test_text.tsv

In [13]:
# Load the validation dataset into a pandas dataframe.
df_t = read_data('./data/test_text.tsv')

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df_t.shape[0]))

# Display 10 random rows from the data.
df_t.sample(10)

Number of test sentences: 13,000



,Id,Text
3413,3413,in this section it sets up the mood for the wh...
9499,9499,"the film is told in flashback, on one such nig..."
12668,12668,kenji thomas jr. has already done his part to ...
6861,6861,a beautiful acrobat - a menacing gorilla - a g...
2203,2203,bread of vengeance begins with a tribute to th...
6227,6227,john mckay and his girlfriend anna duck enter ...
1818,1818,"michale shah, a high class executive at a indu..."
2217,2217,"during a routine combat mission, miles suffers..."
9440,9440,the plot revolves around the vampire giselle d...
11063,11063,"henry roberts, a young student and with his ne..."


In [14]:
# pre-processing for validation set
sentences_t = df_t['Text'].values
sentences_t


array(['the story revolves around a basque roman catholic priest dedicated to committing as many sins as possible , a death metal salesman from carabanchel , and the italian host of a tv',
       "hello all: i'm looking for general feedback. are there confusing, boring, or awkward passages? are the characters compelling? are there scenes that i should decompact? i haven't written much prose, and all",
       '"when did i choose to come here?" he asked as he stood upon the cliff, looking up to the grey sky. it looked like rain. "do you really want to know? "',
       ...,
       'dr. fisher is a student of and an intelligent scientist. his main work consists of the " visualization of a brain which functions through an asperger\'s syndrome. " the story of the',
       'the story begins sixteen years after the events of predator\'s gold, in the grounded ice city of anchorage, now "anchorage-in-vineland". wren natsworthy, the child of tom natsworthy and hester shaw, is bored',
       "spec

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_t = []
attention_masks_t = []

# For every sentence...
for sent in sentences_t:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_t.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_t.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_t = torch.cat(input_ids_t, dim=0)
attention_masks_t = torch.cat(attention_masks_t, dim=0)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids_t, attention_masks_t)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids_t)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask= batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(np.argmax(logits, axis=1).flatten())
  # true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 115,590 test sentences...
    DONE.


In [0]:
print('Positive samples: %d of %d (%.2f%%)' % (df_test['class'].sum(), len(df_test['class']), (df_test['class'].sum() / len(df_test['class']) * 100.0)))

Positive samples: 7217 of 14434 (50.00%)


In [0]:
predictions[0]

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1])

In [0]:
with open('test3.csv','w') as f:
  f.write('Id,Category\n')
  i = -1
  for b in predictions:
    for v in b:
      i += 1
      f.write('{},{}\n'.format(str(i),'human' if v == 1 else 'machine'))

Test on train.tsv

In [17]:
# pre-processing for validation on train.tsv
sentences_test = df['text'].values
labels_test = df['class'].values

labels_test[labels_test == 'machine'] = 0
labels_test[labels_test == 'human'] = 1

labels_test = labels_test.astype(dtype = 'int')

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_test = []
attention_masks_test = []

# For every sentence...
for sent in sentences_test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_test.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_test.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)
labels_test = torch.tensor(labels_test)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_test[0])
print('Token IDs:', input_ids_test[0])

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

test_dataset = TensorDataset(input_ids_test, attention_masks_test, labels_test)

batch_size = 32

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    './model_save_theo/cls_new', # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Original:  once upon a time in the southern chinese city of guancheng, there lived four families, each of them faithful keepers of martial arts. anyone who wanted to establish a new sect, or
Token IDs: tensor([  101,  2320,  2588,  1037,  2051,  1999,  1996,  2670,  2822,  2103,
         1997, 19739,  2319, 21043,  1010,  2045,  2973,  2176,  2945,  1010,
         2169,  1997,  2068, 11633, 24018,  1997,  7761,  2840,  1012,  3087,
         2040,   102])


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
# Prediction on train.csv

print('Predicting labels for {:,} test sentences...'.format(len(input_ids_test)))

# Put model in evaluation mode
model.eval()

total_test_accuracy = 0

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

  total_test_accuracy += flat_accuracy(logits, label_ids)

avg_test_accuracy = total_test_accuracy / len(test_dataloader)

print(avg_test_accuracy)

print('    DONE.')

Predicting labels for 115,590 test sentences...
0.9995502352615555
    DONE.


Test on generated sentences in part 1 

In [0]:
sentences_t = ['My movie tells the story of an old captain, and his last days as a general of the indian army during world war i. he begins a long struggle for survival',
'The film opens with a limousine driver, jay, with his friend and potential mate, riding in it. jay agrees to a simple plan of the day and throws himself',
'One child has been able to learn a wonderful thing, but he can do no more than act like it does not matter. it is part of the cycle of agony',
"When she comes out, aisha's life is changed forever. from the first day of school to cheerleading lessons, ajay is devastated. he believes that time",
"If his bet is a winner, the cowboy and the ranch hand then decide to find out what kind of rancher roy mcewan might be. after spending years",
"Our princess starts out the cat and mouse game, that everyone on the planet is looking for. some people accidentally bump into. at this point, i guess some guy",
'First grown from the quadriplegic youth, alrel and his team of friends stop at the birth canal station, a nearby field called " the express lane "',
"Natural french gardener mr. neville has a slip, and without the money he had been promised the previous year by colonel sieine, well, the pipe mr neville",
"We go out of the garden and to the house. we are seen taking up their bows, filling them up with water. they then shoot, and kill someone who",
"Because of his luck, anthony delantan, a popular and intense writer, has taken to becoming an astronaut after the 70s. already, he has gained influence"]

In [0]:
input_ids_t = []
attention_masks_t = []

# For every sentence...
for sent in sentences_t:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_t.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_t.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids_t = torch.cat(input_ids_t, dim=0)
attention_masks_t = torch.cat(attention_masks_t, dim=0)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids_t, attention_masks_t)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [0]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids_t)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask= batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(np.argmax(logits, axis=1).flatten())
  # true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 10 test sentences...
    DONE.


In [0]:
print(predictions)

[array([1, 1, 0, 1, 1, 1, 0, 1, 0, 0])]


In [0]:
print(', '.join(['Human' if i else 'Machine' for i in predictions[0]]))

Human, Human, Machine, Human, Human, Human, Machine, Human, Machine, Machine
